<a href="https://colab.research.google.com/github/AliSalman41/PV-anomaly-detection/blob/main/Annotation_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p workspaceFolder # creating main workspace folder

In [ ]:
import os
import json
%pip install kaggle
%cd /content/workspaceFolder/

os.environ['KAGGLE_CONFIG_DIR'] = "/content/workspaceFolder/"

In [ ]:
# downloading from kaggle requires API key - get from account details
from getpass import getpass
user = getpass("user:\n")
key = getpass("key:\n")

env = {"username":user, "key":key}
with open('kaggle.json', 'w') as outfileKey:
  json.dump(env, outfileKey)
!chmod 600 'kaggle.json'

In [ ]:
# unzip kaggle dataset
import zipfile
!kaggle datasets download -d marcosgabriel/photovoltaic-system-thermography
!unzip /content/workspaceFolder/photovoltaic-system-thermography.zip -d /content/workspaceFolder/datasetRaw/

In [ ]:
# create new folder for new annotations
%cd -q '/content/workspaceFolder/'
!mkdir -p newData/annotations

In [ ]:
# duplicate annotations into new folder
from distutils.dir_util import copy_tree
copy_tree("datasetRaw/annotations", "newData/annotations")

In [ ]:
%cd -q '/content/workspaceFolder/'

In [ ]:
# open each of the old annotation files, find appropriate data, and rewrite new corresponding file
dir = '/content/workspaceFolder/datasetRaw/annotations/'
newDir = '/content/workspaceFolder/newData/annotations/'
print(dir, newDir)

for fileName in os.listdir(dir):
  %cd -q '/content/workspaceFolder/datasetRaw/annotations/'
  f = open(fileName)
  data = json.loads(f.read())
  boxes = [{"label":"", "x":0, "y":0, "width":0, "height":0} for i in range (len(data["instances"]))]

  print("# instances in ", f.name, " : ", len(boxes))

  final = {"boxes":[], "height":512, "key":(os.path.splitext(f.name)[0] + ".jpg"), "width":640}
  for i in range(len(data["instances"])):
    boxes[i]["label"]="PV_module"
    x, y = [], []
    boxes[i]["x"], boxes[i]["y"] = 0, 0
    for j in range(0, 4):
      x.append(data["instances"][i]["corners"][j]["x"])
      y.append(data["instances"][i]["corners"][j]["y"])
      boxes[i]["x"], boxes[i]["y"] = max(x), max(y)
    boxes[i]["width"], boxes[i]["height"] = max(x) - min(x), max(y) - min(y)
    final["boxes"].append(boxes[i])
  %cd -q '/content/workspaceFolder/newData/annotations/'
  with open(fileName, "w") as outfile:
    json.dump(final, outfile)
  f.close()
  outfile.close()
  

In [ ]:
# zip new annotations into "file.zip"
!zip -r /content/workspaceFolder/file.zip /content/workspaceFolder/newData/annotations/

In [ ]:
# download new annotations
from google.colab import files
files.download("/content/workspaceFolder/file.zip")